In [2]:
pip install skfolio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.0/734.0 kB 13.9 MB/s eta 0:00:00


In [3]:
import numpy as np
from plotly.io import show
from sklearn.model_selection import train_test_split

from skfolio import Population, RiskMeasure
from skfolio.datasets import load_sp500_dataset
from skfolio.preprocessing import prices_to_returns

In [4]:
prices = load_sp500_dataset()
prices.head()

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,
1990-01-02,0.264,4.125,4.599,0.144,4.991,14.391,1.117,3.438,3.394,2.235,6.658,4.215,0.384,4.738,1.021,3.860,3.322,0.310,3.653,4.068
1990-01-03,0.266,4.000,4.636,0.161,4.910,14.364,1.121,3.452,3.508,2.203,6.658,4.282,0.386,4.692,1.024,3.853,3.322,0.304,3.653,4.027
1990-01-04,0.267,3.938,4.537,0.159,4.847,14.283,1.128,3.459,3.522,2.192,6.621,4.215,0.397,4.646,1.041,3.777,3.322,0.301,3.634,3.987
1990-01-05,0.268,3.812,4.438,0.159,4.775,14.148,1.113,3.423,3.536,2.174,6.549,4.128,0.387,4.581,1.032,3.709,3.322,0.288,3.595,3.966
1990-01-08,0.269,3.812,4.463,0.147,4.820,14.229,1.102,3.481,3.536,2.220,6.549,4.181,0.393,4.664,1.023,3.777,3.322,0.282,3.644,4.027


In [26]:
X = prices_to_returns(prices)
X_train, X_test = train_test_split(X, test_size=0.33, shuffle=False)
print(X_train.head())

                AAPL       AMD       BAC       BBY       CVX        GE  \
Date                                                                     
1990-01-03  0.007576 -0.030303  0.008045  0.118056 -0.016229 -0.001876   
1990-01-04  0.003759 -0.015500 -0.021355 -0.012422 -0.012831 -0.005639   
1990-01-05  0.003745 -0.031996 -0.021821  0.000000 -0.014855 -0.009452   
1990-01-08  0.003731  0.000000  0.005633 -0.075472  0.009424  0.005725   
1990-01-09 -0.007435  0.016527  0.000000  0.000000 -0.007469 -0.020803   

                  HD       JNJ       JPM        KO       LLY       MRK  \
Date                                                                     
1990-01-03  0.003581  0.004072  0.033589 -0.014318  0.000000  0.015896   
1990-01-04  0.006244  0.002028  0.003991 -0.004993 -0.005557 -0.015647   
1990-01-05 -0.013298 -0.010408  0.003975 -0.008212 -0.010874 -0.020641   
1990-01-08 -0.009883  0.016944  0.000000  0.021159  0.000000  0.012839   
1990-01-09 -0.026316 -0.031026 -0.031

In [27]:
from skfolio.optimization import MeanRisk ,ObjectiveFunction
function = MeanRisk(
    risk_measure=RiskMeasure.CVAR,
    objective_function=ObjectiveFunction.MINIMIZE_RISK,
    portfolio_params=dict(name="Min CVaR"),
)
function.fit(X_train)
function.weights_

array([2.16562529e-02, 1.41742527e-10, 3.64888981e-11, 1.47358386e-02,
       1.35927671e-01, 1.91035768e-10, 3.58350612e-10, 2.10319630e-01,
       4.89793803e-11, 8.14733606e-02, 1.92817471e-02, 4.13607836e-10,
       7.79844555e-10, 1.26155697e-01, 4.04585759e-10, 1.52708811e-01,
       1.20106233e-02, 6.41726679e-03, 1.01024013e-01, 1.18289085e-01])

In [28]:
function = MeanRisk(budget=None, min_budget=0.3, max_budget=0.5)
function.fit(X)
print(sum(function.weights_))
function.weights_

0.30000003916225304


array([7.82679381e-03, 1.18331943e-07, 1.14395653e-07, 2.05218577e-03,
       2.15009010e-02, 2.14359256e-07, 2.58606661e-07, 5.94038815e-02,
       1.38502369e-07, 3.62601668e-02, 9.04209387e-03, 6.30979655e-03,
       2.96944698e-03, 3.41087755e-02, 3.45801015e-03, 4.97522144e-02,
       2.94825754e-03, 2.87672749e-03, 3.47226412e-02, 2.67673024e-02])

In [31]:
import numpy as np
import cvxpy as cp
import pandas as pd
weights = cp.Variable(X_train.shape[1])
X_train = X_train.values if isinstance(X_train, pd.DataFrame) else X_train

#Risk Measures
alpha = 0.95
VaR = cp.Variable()
z = cp.Variable(X_train.shape[0])
risk_measure = VaR + (1 / (1 - alpha)) * cp.sum(z) / X_train.shape[0]

#Objective Function
objective = cp.Minimize(risk_measure)

#Constraints
constraints = [
    weights >= 0,
    cp.sum(weights) == 1,
    z >= 0,
    z >= -X_train @ weights - VaR
]

#Optimization problem
problem = cp.Problem(objective, constraints)

# Check if the problem is convex
is_convex = problem.is_dcp()

In [32]:
print(f"Is the MeanRisk model with CVaR risk measure and MINIMIZE_RISK objective function convex? {is_convex}")

Is the MeanRisk model with CVaR risk measure and MINIMIZE_RISK objective function convex? True


In [20]:
# Non-Linear Constraints
# Add a non-convex constraint to the weights, such as an exponential constraint
non_convex_constraint = cp.sum(cp.log(weights + 1e-6)) <= 2  # Non-convex constraint example

# Constraints
nc_constraints = [
    weights >= 0,
    cp.sum(weights) == 1,
    z >= 0,
    z >= -X_train @ weights - VaR,
    non_convex_constraint  # Adding the non-convex constraint
]

# Optimization problem
nc_problem = cp.Problem(objective, nc_constraints)

# Check if the problem is convex
is_convex = nc_problem.is_dcp()
print(f"Is the modified MeanRisk model with CVaR risk measure and MINIMIZE_RISK objective function convex? {is_convex}")

Is the modified MeanRisk model with CVaR risk measure and MINIMIZE_RISK objective function convex? False


In [33]:
from skfolio.optimization import EqualWeighted, MeanRisk, ObjectiveFunction
benchmark = EqualWeighted(portfolio_params=dict(name="Equal Weighted"))
# Even if `X` has no impact (as it is equal weighted), we still need to call `fit` for
# API consistency.
benchmark.fit(X_train)
benchmark.weights_

array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])

In [34]:
X_test = X_test.values if isinstance(X_test, pd.DataFrame) else X_test
pred_model = function.predict(X_test)
pred_bench = benchmark.predict(X_test)

In [35]:
np.asarray(pred_model)


array([ 0.00071098,  0.00101054, -0.00025058, ...,  0.00248715,
        0.00123894, -0.0034885 ])

In [36]:
print(pred_model.cvar)
print(pred_bench.cvar)

0.006270742895541362
0.025061083134673378


In [37]:
population = Population([pred_model, pred_bench])

In [38]:
population.plot_composition()